# Project 6: IMDB

This project involves NLP, decision trees, bagging, boosting, and more!

---

## Load packages

You are likely going to need to install the `imdbpie` package:

    > pip install imdbpie

---

In [4]:
import os
import subprocess
import collections
import re
import csv
import json

import pandas as pd
import numpy as np
import scipy

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import psycopg2
import requests
from imdbpie import Imdb
import nltk

import urllib
from bs4 import BeautifulSoup
import nltk

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

---

## Part 1: Acquire the Data

You will connect to the IMDB API to query for movies. 

See here for documentation on how to use the package:

https://github.com/richardasaurus/imdb-pie

#### 1. Connect to the IMDB API

In [5]:
imdb = Imdb()
imdb = Imdb(anonymize=True)

#### 2. Query the top 250 rated movies in the database

In [6]:
imdb.top_250()

[{u'can_rate': True,
  u'image': {u'height': 1388,
   u'url': u'http://ia.media-imdb.com/images/M/MV5BODU4MjU4NjIwNl5BMl5BanBnXkFtZTgwMDU2MjEyMDE@._V1_.jpg',
   u'width': 933},
  u'num_votes': 1659063,
  u'rating': 9.3,
  u'tconst': u'tt0111161',
  u'title': u'The Shawshank Redemption',
  u'type': u'feature',
  u'year': u'1994'},
 {u'can_rate': True,
  u'image': {u'height': 500,
   u'url': u'http://ia.media-imdb.com/images/M/MV5BMjEyMjcyNDI4MF5BMl5BanBnXkFtZTcwMDA5Mzg3OA@@._V1_.jpg',
   u'width': 333},
  u'num_votes': 1135585,
  u'rating': 9.2,
  u'tconst': u'tt0068646',
  u'title': u'The Godfather',
  u'type': u'feature',
  u'year': u'1972'},
 {u'can_rate': True,
  u'image': {u'height': 500,
   u'url': u'http://ia.media-imdb.com/images/M/MV5BNDc2NTM3MzU1Nl5BMl5BanBnXkFtZTcwMTA5Mzg3OA@@._V1_.jpg',
   u'width': 333},
  u'num_votes': 775548,
  u'rating': 9,
  u'tconst': u'tt0071562',
  u'title': u'The Godfather: Part II',
  u'type': u'feature',
  u'year': u'1974'},
 {u'can_rate': True,
 

#### 3. Make a dataframe from the movie data

Keep the fields:

    num_votes
    rating
    tconst
    title
    year
    
And discard the rest

In [7]:
movies = pd.DataFrame(imdb.top_250(), columns = ['num_votes', 'rating', 'tconst', 'title', 'year'])

In [8]:
movies.head()

,num_votes,rating,tconst,title,year
0,1659063,9.3,tt0111161,The Shawshank Redemption,1994
1,1135585,9.2,tt0068646,The Godfather,1972
2,775548,9.0,tt0071562,The Godfather: Part II,1974
3,1644778,9.0,tt0468569,The Dark Knight,2008
4,848733,8.9,tt0108052,Schindler's List,1993


#### 3. Select only the top 100 movies

In [9]:
sorted(movies.rating.head(100), reverse = True)

[9.3000000000000007,
 9.1999999999999993,
 9.0,
 9.0,
 8.9000000000000004,
 8.9000000000000004,
 8.9000000000000004,
 8.9000000000000004,
 8.9000000000000004,
 8.9000000000000004,
 8.8000000000000007,
 8.8000000000000007,
 8.8000000000000007,
 8.8000000000000007,
 8.6999999999999993,
 8.6999999999999993,
 8.6999999999999993,
 8.6999999999999993,
 8.6999999999999993,
 8.6999999999999993,
 8.6999999999999993,
 8.5999999999999996,
 8.5999999999999996,
 8.5999999999999996,
 8.5999999999999996,
 8.5999999999999996,
 8.5999999999999996,
 8.5999999999999996,
 8.5999999999999996,
 8.5999999999999996,
 8.5999999999999996,
 8.5999999999999996,
 8.5999999999999996,
 8.5999999999999996,
 8.5999999999999996,
 8.5999999999999996,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.4000000000000004,
 8.4000000000000004,
 8.4000000000000004,
 8.4000000000000004,
 8.4000000000000004,
 8.4000000000000004,
 8.4

#### 4. Get the genres and runtime for each movie and add them to the dataframe

There can be multiple genres per movie, so this will need some finessing.

In [10]:
genres = []

for tconst in movies.tconst:
    title = imdb.get_title_by_id(tconst)
    genres.append(title.genres)
    
print genres

[[u'Crime', u'Drama'], [u'Crime', u'Drama'], [u'Crime', u'Drama'], [u'Action', u'Crime', u'Thriller'], [u'Biography', u'Drama', u'History'], [u'Crime', u'Drama'], [u'Crime', u'Drama'], [u'Adventure', u'Drama', u'Fantasy'], [u'Western'], [u'Drama'], [u'Adventure', u'Drama', u'Fantasy'], [u'Action', u'Adventure', u'Fantasy', u'Sci-Fi'], [u'Drama', u'Romance'], [u'Action', u'Mystery', u'Sci-Fi', u'Thriller'], [u'Action', u'Adventure', u'Drama', u'Fantasy'], [u'Drama'], [u'Biography', u'Crime', u'Drama'], [u'Action', u'Sci-Fi'], [u'Action', u'Adventure', u'Drama'], [u'Action', u'Adventure', u'Fantasy', u'Sci-Fi'], [u'Crime', u'Drama'], [u'Crime', u'Drama', u'Mystery', u'Thriller'], [u'Crime', u'Drama', u'Thriller'], [u'Drama', u'Family', u'Fantasy', u'Romance'], [u'Crime', u'Drama', u'Mystery', u'Thriller'], [u'Comedy', u'Drama', u'Romance', u'War'], [u'Crime', u'Drama', u'Thriller'], [u'Western'], [u'Animation', u'Adventure', u'Family', u'Fantasy'], [u'Action', u'Drama', u'War'], [u'Crime

In [11]:
movies['genre'] = genres

In [ ]:
movies.head()

In [13]:
runtimes = []

for tconst in movies.tconst:
    title = imdb.get_title_by_id(tconst)
    runtime = round(title.runtime / 3600., 2) # Dividing by 3600 to change seconds to hours and rounding it to 2 decimals
    runtimes.append(runtime)

# title = imdb.get_title_by_id('tt0111161')
# title.runtime / 3600.
print runtimes

[2.37, 2.92, 3.37, 2.53, 3.25, 1.6, 2.57, 3.35, 2.68, 2.32, 2.97, 2.07, 2.37, 2.47, 2.98, 2.22, 2.43, 2.27, 2.63, 2.02, 2.17, 2.12, 1.97, 2.17, 1.77, 1.93, 1.83, 2.42, 2.08, 2.82, 1.68, 2.82, 1.7, 1.82, 1.45, 1.92, 1.87, 1.87, 1.45, 3.15, 2.55, 2.5, 2.52, 1.93, 1.78, 2.58, 1.88, 2.55, 2.17, 1.48, 1.58, 1.83, 1.95, 2.08, 2.28, 2.58, 2.75, 2.4, 1.47, 1.48, 1.63, 2.73, 2.03, 2.57, 2.23, 2.0, 1.98, 2.27, 2.32, 3.98, 2.15, 2.18, 1.93, 1.65, 1.65, 2.03, 2.97, 1.7, 2.27, 1.88, 1.78, 1.72, 2.15, 3.78, 1.8, 1.93, 2.67, 2.15, 2.48, 1.48, 1.72, 1.7, 1.35, 1.52, 2.55, 1.13, 2.3, 2.2, 2.08, 1.47, 2.12, 2.83, 2.05, 1.9, 2.3, 1.83, 1.48, 2.1, 2.33, 1.92, 2.83, 2.0, 2.1, 2.18, 1.6, 1.55, 2.15, 1.58, 2.6, 2.18, 2.87, 2.17, 2.45, 2.83, 1.97, 1.97, 1.8, 1.43, 2.7, 2.38, 2.15, 2.68, 1.2, 2.27, 2.15, 1.6, 1.57, 1.95, 1.78, 2.27, 1.98, 2.97, 1.25, 1.52, 2.95, 2.07, 2.33, 3.0, 2.25, 3.1, 2.2, 1.8, 1.93, 1.95, 2.32, 3.05, 2.17, 1.63, 1.57, 3.97, 1.75, 2.1, 1.63, 1.78, 1.67, 1.82, 2.03, 2.47, 1.53, 2.48, 1.85,

In [14]:
movies['runtime'] = runtimes

In [15]:
movies.head()

,num_votes,rating,tconst,title,year,genre,runtime
0,1659063,9.3,tt0111161,The Shawshank Redemption,1994,"[Crime, Drama]",2.37
1,1135585,9.2,tt0068646,The Godfather,1972,"[Crime, Drama]",2.92
2,775548,9.0,tt0071562,The Godfather: Part II,1974,"[Crime, Drama]",3.37
3,1644778,9.0,tt0468569,The Dark Knight,2008,"[Action, Crime, Thriller]",2.53
4,848733,8.9,tt0108052,Schindler's List,1993,"[Biography, Drama, History]",3.25


#### 4. Write the Results to a csv

In [16]:
movies.to_csv('movie_csv', sep = '\t', encoding = 'utf-8')

---

## Part 2: Wrangle the text data

#### 1. Scrape the reviews for the top 100 movies

*Hint*: Use a loop to scrape each page at once

In [17]:
import requests
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

url = "http://www.imdb.com/chart/top"
response = requests.get(url)

# Pull HTML string out of requests
HTML = response.text


ImportError: No module named scrapy.selector

In [83]:
# The same thing but "absolute" reference
hrefs = Selector(text=HTML).xpath('//td[@class="titleColumn"]/a[@href]/@href').extract()
print hrefs

[u'/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=0EFWZTAGYSPXPMSTWFAV&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1', u'/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=0EFWZTAGYSPXPMSTWFAV&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_2', u'/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=0EFWZTAGYSPXPMSTWFAV&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_3', u'/title/tt0468569/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=0EFWZTAGYSPXPMSTWFAV&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_4', u'/title/tt0108052/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=0EFWZTAGYSPXPMSTWFAV&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_5', u'/title/tt0050083/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=0EFWZTAGYSPXPMSTWFAV&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_6', u'/title/tt0110912/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=0EFWZTAGYSPXPMSTW

In [18]:
reviews = []
import urllib
for i in movies.tconst:
    path = 'http://www.imdb.com/title/%s/reviews?ref_=tt_urv' % i
    r = urllib.urlopen(path).read()
    soup = BeautifulSoup(r)
    reviews.append(soup)

C:\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [19]:
reviews

[<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="unicode-escape"/>\n<title></title>\n</head>\n<body>\n<br/>\n<h1 align="center">Access denied<br/>\n<small>http: 403</small></h1>\n<p>Sorry, this imdb.com (Internet Movie Database) page is not\naccessible from your address or browser. This page contains\ninformation which may explain why you are seeing this error instead of\nthe page you wanted. Please read this information and if <b>after</b>\nreading it you still do not understand why you have been denied access\nthen you should use the contact details at the end to request\ninformation specific to your case.</p>\n<p>Note: the majority of people seeing this page are <b>innocent</b>\nusers suffering because of offenders who use the same Internet address\nor software as they do. Chances are, you're not the offender and we'll\ndo our best to restore your access once we're aware of your\ncircumstances.</p>\n<hr width="30%"/>\n<p>The three main reasons for seeing this message are:<

#### 2. Extract the reviews and the rating per review for each movie

*Note*: "soup" from BeautifulSoup is the html returned from all 25 pages. You'll need to either address each page individually or break them down by elements.

#### 3. Remove the non AlphaNumeric characters from reviews

#### 4. Calculate the top 200 ngrams from the user reviews

Use the `TfidfVectorizer` in sklearn.

Recommended parameters:

    ngram_range = (1, 2)
    stop_words = 'english'
    binary = False
    max_features = 200

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### 5. Merge the user reviews and ratings

#### 6. Save this merged dataframe as a csv

---

## Part 3: Combine Tables in PostgreSQL

#### 1. Import your two .csv data files into your Postgre Database as two different tables

For ease, we can call these table1 and table2

#### 2. Connect to database and query the joined set

#### 3. Join the two tables 

#### 4. Select the newly joined table and save two copies of the into dataframes

---

## Part 4: Parsing and Exploratory Data Analysis

#### 1. Rename any columns you think should be renamed for clarity

#### 2. Describe anything interesting or suspicious about your data (quality assurance)

#### 3. Make four visualizations of interest to you using the data

---

## Part 5: Decision Tree Classifiers and Regressors

#### 1. What is our target attribute? 

Choose a target variable for the decision tree regressor and the classifier. 

#### 2. Prepare the X and Y matrices and preprocess data as you see fit

#### 3. Build and cross-validate your decision tree classifier

#### 4. Gridsearch optimal parameters for your classifier. Does the performance improve?

#### 5. Build and cross-validate your decision tree regressor

#### 6. Gridsearch the optimal parameters for your classifier. Does performance improve?

---

## Part 6: Elastic Net


#### 1. Gridsearch optimal parameters for an ElasticNet using the regression target and predictors you used for the decision tree regressor.


#### 2. Is cross-validated performance better or worse than with the decision trees? 

#### 3. Explain why the elastic net may have performed best at that particular l1_ratio and alpha

---

## Part 7: Bagging and Boosting: Random Forests, Extra Trees, and AdaBoost

#### 1. Load the random forest regressor, extra trees regressor, and adaboost regressor from sklearn

#### 2. Gridsearch optimal parameters for the three different ensemble methods.

#### 3. Evaluate the performance of the two bagging and one boosting model. Which performs best?

#### 4. Extract the feature importances from the Random Forest regressor and make a DataFrame pairing variable names with their variable importances.

#### 5. Plot the ranked feature importances.

#### 6.1 [BONUS] Gridsearch an optimal Lasso model and use it for variable selection (make a new predictor matrix with only the variables not zeroed out by the Lasso). 

#### 6.2 [BONUS] Gridsearch your best performing bagging/boosting model from above with the features retained after the Lasso. Does the score improve?

#### 7.1. [BONUS] Select a threshold for variable importance from your Random Forest regressor and use that to perform feature selection, creating a new subset predictor matrix.

#### 7.2 [BONUS] Using BaggingRegressor with a base estimator of your choice, test a model using the feature-selected dataset you made in 7.1

---

## [VERY BONUS] Part 8: PCA

#### 1. Perform a PCA on your predictor matrix

#### 2. Examine the variance explained and determine what components you want to keep based on them.

#### 3. Plot the cumulative variance explained by the ordered principal components.

#### 4. Gridsearch an elastic net using the principal components you selected as your predictors. Does this perform better than the elastic net you fit earlier?

#### 5. Gridsearch a bagging ensemble estimator that you fit before, this time using the principal components as predictors. Does this perform better or worse than the original? 

#### 6. Look at the loadings of the original predictor columns on the first 3 principal components. Is there any kind of intuitive meaning here?

Hint, you will probably want to sort by absolute value of magnitude of loading, and also only look at the obviously important (larger) ones!

# [Extremely Bonus] Part 9:  Clustering

![](https://snag.gy/jPSZ6U.jpg)

 ***Bonus Bonus:***
This extended bonus question is asking to do something we never really talked about but would like for you to attempt based on the assumptions that we learned during this weeks clustering lesson(s).

#### 1. Import your favorite clustering module

#### 2. Encode categoricals

#### 3. Evaluate cluster metics solely based on a range of K
If K-Means:  SSE/Inertia vs Silhouette (ie: Elbow), silhouette average, etc

#### 4.  Look at your data based on the subset of your predicted clusters.
Assign the cluster predictions back to your dataframe in order to see them in context.  This is great to be able to group by cluster to get a sense of the data that clumped together.

#### 5. Describe your findings based on the predicted clusters 
_How well did it do?  What's good or bad?  How would you improve this? Does any of it make sense?_